# Trabajo Práctico Nro. 1 
### Ing. Javier Ouret - 2025 - UCA - Facultad de Ingeniería
## Ejemplos de Servidores TCP y Clientes TCP usando bash kernel para su compilación.
## A) Servidor Simple y Servidor Concurrente

**NOTA: Cerrar las ventanas de las Terminales al finalizar su uso**

## Servidor TCP

Envío el código a un archivo:

In [1]:
echo '#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <netinet/in.h>
#include <arpa/inet.h>

#define MAX 1024
#define PORT 8080

int main() {
    int server_fd, client_fd;
    struct sockaddr_in server_addr, client_addr;
    char buffer[MAX];
    char client_name[50];

    // Crear socket
    server_fd = socket(AF_INET, SOCK_STREAM, 0);
    if (server_fd == -1) {
        perror("Socket no pudo abrirse ... error ...");
        exit(1);
    }

    // Configurar estructura
    server_addr.sin_family = AF_INET;
    server_addr.sin_addr.s_addr = INADDR_ANY;
    server_addr.sin_port = htons(PORT);
    memset(&(server_addr.sin_zero), 0, 8);

    // Asociar dirección y puerto
    if (bind(server_fd, (struct sockaddr*)&server_addr, sizeof(server_addr)) != 0) {
        perror("Bind falló");
        exit(1);
    }

    // Escuchar conexiones
    if (listen(server_fd, 1) != 0) {
        perror("Listen con error...");
        exit(1);
    }

    printf("Servidor esperando conexión en el puerto %d...\n", PORT);
    socklen_t addr_size = sizeof(client_addr);
    client_fd = accept(server_fd, (struct sockaddr*)&client_addr, &addr_size);
    if (client_fd < 0) {
        perror("Falló al aceptar conexión");
        exit(1);
    }

    // Recibir nombre del cliente
    recv(client_fd, client_name, sizeof(client_name), 0);
    printf("Cliente conectado: %s\n", client_name);

    while (1) {
        // Recibir mensaje del cliente
        memset(buffer, 0, MAX);
        int bytes = recv(client_fd, buffer, MAX, 0);
        if (bytes <= 0) {
            printf("Cliente desconectado.\n");
            break;
        }

        if (strncmp(buffer, "FIN", 3) == 0) {
            printf("%s ha finalizado la conexión.\n", client_name);
            break;
        }

        printf("%s: %s", client_name, buffer);

        // Enviar respuesta
        printf("Servidor: ");
        memset(buffer, 0, MAX);
        fgets(buffer, MAX, stdin);
        send(client_fd, buffer, strlen(buffer), 0);

        if (strncmp(buffer, "FIN", 3) == 0) {
            printf("Servidor finaliza la conexión.\n");
            break;
        }
    }

    close(client_fd);
    close(server_fd);
    return 0;
}
' > servidor_tcp_bash.c

Compilo el código usando Bash C:

In [2]:
gcc --version

gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
gcc servidor_tcp_bash.c -o servidor_tcp_bash

Verifico que el ejecutable esté disponible:

In [4]:
ls

 cliente                                servidor
 cliente.c                              servidor.c
'Cliente Servidor TCP con bash.ipynb'   servidor_tcp_bash
 cliente_tcp_bash                       servidor_tcp_bash.c
 cliente_tcp_bash.c                     servidor_tcp_c.ipynb
 cliente_tcp_c.ipynb                    servidor_tcp_concurrente_bash
'prueba de C con bash.ipynb'            servidor_tcp_concurrente_bash.c
'prueba de C.ipynb'


La terminal de Jupyter puede funcionar mal. Para ejecutarlo correctamente abro ventana por afuera del Jupyter Notebook:

In [5]:
gnome-terminal -- bash -c "./servidor_tcp_bash; exec bash"

## Cliente TCP

Envío el código a un archivo:

In [6]:
echo '#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <netinet/in.h>
#include <arpa/inet.h>

#define MAX 1024
#define PORT 8080

int main() {
    int sockfd;
    struct sockaddr_in server_addr;
    char buffer[MAX];
    char name[50];

    // Crear socket
    sockfd = socket(AF_INET, SOCK_STREAM, 0);
    if (sockfd == -1) {
        perror("Socket falló");
        exit(1);
    }

    server_addr.sin_family = AF_INET;
    server_addr.sin_port = htons(PORT);
    inet_pton(AF_INET, "127.0.0.1", &server_addr.sin_addr);
    memset(&(server_addr.sin_zero), 0, 8);

    // Conectar al servidor
    if (connect(sockfd, (struct sockaddr*)&server_addr, sizeof(server_addr)) != 0) {
        perror("Conexión falló");
        exit(1);
    }

    // Enviar nombre
    printf("Ingresar nombre: ");
    fgets(name, sizeof(name), stdin);
    name[strcspn(name, "\n")] = 0; // Eliminar salto de línea
    send(sockfd, name, strlen(name), 0);

    while (1) {
        // Enviar mensaje
        printf("%s: ", name);
        memset(buffer, 0, MAX);
        fgets(buffer, MAX, stdin);
        send(sockfd, buffer, strlen(buffer), 0);

        if (strncmp(buffer, "FIN", 3) == 0) {
            printf("Terminando la conexión...\n");
            break;
        }

        // Recibir respuesta
        memset(buffer, 0, MAX);
        int bytes = recv(sockfd, buffer, MAX, 0);
        if (bytes <= 0 || strncmp(buffer, "FIN", 3) == 0) {
            printf("Servidor finalizó la conexión.\n");
            break;
        }

        printf("Servidor: %s", buffer);
    }

    close(sockfd);
    return 0;
}' > cliente_tcp_bash.c

Compilo el código usando Bash C:

In [7]:
gcc --version

gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



gcc cliente_tcp_bash.c -o cliente_tcp_bash

Verifico que el ejecutable esté disponible:

In [8]:
ls

 cliente                                servidor
 cliente.c                              servidor.c
'Cliente Servidor TCP con bash.ipynb'   servidor_tcp_bash
 cliente_tcp_bash                       servidor_tcp_bash.c
 cliente_tcp_bash.c                     servidor_tcp_c.ipynb
 cliente_tcp_c.ipynb                    servidor_tcp_concurrente_bash
'prueba de C con bash.ipynb'            servidor_tcp_concurrente_bash.c
'prueba de C.ipynb'


Verifico si funciona:

Para ejecutarlo correctamente abro ventana terminal por afuera del Jupyter Notebook (esta ventana permanecerá abierta):

In [17]:
gnome-terminal -- bash -c "./cliente_tcp_bash; exec bash"

Si abro la ventana terminal de esta forma se cerrará al finalizar la ejecución del cliente:

In [18]:
gnome-terminal -- bash -c "./cliente_tcp_bash; echo 'Presionar ENTER para cerrar...'; read"

## Servidor TCP Concurrente

In [ ]:
echo '#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include <signal.h>

#define MAX 1024
#define PORT 8080

void handle_client(int client_fd, struct sockaddr_in client_addr) {
    char buffer[MAX];
    char name[50] = "Cliente";
    char client_ip[INET_ADDRSTRLEN];
    char response[MAX + 100];
    pid_t pid = getpid();

    inet_ntop(AF_INET, &(client_addr.sin_addr), client_ip, INET_ADDRSTRLEN);
    int client_port = ntohs(client_addr.sin_port);

    printf("[PID %d] Conexión desde %s:%d\n", pid, client_ip, client_port);

    // Recibir nombre del cliente
    recv(client_fd, name, sizeof(name), 0);
    printf("[PID %d] Cliente: %s\n", pid, name);

    while (1) {
        memset(buffer, 0, MAX);
        int bytes = recv(client_fd, buffer, MAX, 0);
        if (bytes <= 0) {
            printf("[PID %d] %s se desconectó.\n", pid, name);
            break;
        }

        // Verificar si cliente quiere finalizar
        if (strncmp(buffer, "FIN", 3) == 0) {
            printf("[PID %d] %s finalizó la conexión.\n", pid, name);
            break;
        }

        printf("[%s | PID %d]: %s", name, pid, buffer);

        // Preparar respuesta automática
        snprintf(response, sizeof(response),
                 "Este es el mensaje recibido por el servidor: %s", buffer);
        send(client_fd, response, strlen(response), 0);
    }

    close(client_fd);
    printf("[PID %d] Proceso hijo termina.\n", pid);
    exit(0);
}

int main() {
    int server_fd, client_fd;
    struct sockaddr_in server_addr, client_addr;

    signal(SIGCHLD, SIG_IGN); // Evita procesos zombies

    server_fd = socket(AF_INET, SOCK_STREAM, 0);
    if (server_fd == -1) {
        perror("Error al crear socket");
        exit(1);
    }

    server_addr.sin_family = AF_INET;
    server_addr.sin_addr.s_addr = INADDR_ANY;
    server_addr.sin_port = htons(PORT);
    memset(&(server_addr.sin_zero), 0, 8);

    if (bind(server_fd, (struct sockaddr*)&server_addr, sizeof(server_addr)) != 0) {
        perror("Error en bind");
        exit(1);
    }

    if (listen(server_fd, 5) != 0) {
        perror("Error en listen");
        exit(1);
    }

    printf("Servidor concurrente activo en puerto %d...\n", PORT);

    socklen_t len = sizeof(client_addr);
    while (1) {
        client_fd = accept(server_fd, (struct sockaddr*)&client_addr, &len);
        if (client_fd < 0) {
            perror("Fallo en accept");
            continue;
        }

        pid_t pid = fork();
        if (pid == 0) {
            close(server_fd);
            handle_client(client_fd, client_addr);
        } else if (pid > 0) {
            printf("[PID %d] Cliente conectado. Proceso hijo PID %d\n", getpid(), pid);
            close(client_fd);
        } else {
            perror("Error en fork");
            close(client_fd);
        }
    }

    close(server_fd);
    return 0;
}' > servidor_tcp_concurrente_bash.c

Compilo el código usando Bash C y verifico que el ejecutable esté disponible

In [ ]:
gcc servidor_tcp_concurrente_bash.c -o servidor_tcp_concurrente_bash | ls


In [ ]:
gnome-terminal -- bash -c "./servidor_tcp_concurrente_bash; echo 'Presionar ENTER para cerrar...'; read"

**Ejercicios a realizar durante la clase. Incluir los códigos dentro de este mismo Notebook**

A-1) Convertir el servidor simple a servidor iterativo.   
A-2) Convertir el cliente y el servidor a un cliente UDP que utilice connect() y un servidor iterativo con UDP.   
A-3) Mejorar la aplicación de chat para que identifique a los clientes conectados.   
A-4) Abrir varios clientes y hacer que los clientes manden mensajes en forma automática al servidor concurrente para observar el comportamiento del fork().   

## B) Servidores con concurrencia aparente usando select()

¿Qué hace select()?
select() permite monitorear múltiples sockets (conexiones de red), archivos, o entradas estándar (stdin) y saber cuál de ellos está listo para lectura, escritura o tiene errores. Se usa en servidores que manejan múltiples clientes al mismo tiempo. Permite manejar múltiples clientes sin necesidad de hilos (threads) o procesos hijos.

## Servidor con select()

In [ ]:
echo '
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <errno.h>
#include <sys/socket.h>
#include <sys/types.h>
#include <arpa/inet.h>
#include <netinet/in.h>
#include <sys/select.h>

#define PORT 12345
#define MAX_CLIENTS 10
#define BUFFER_SIZE 1024

int main() {
    int server_fd, new_socket, client_sockets[MAX_CLIENTS];
    struct sockaddr_in address;
    fd_set read_fds;
    int max_sd, sd, activity, valread;
    char buffer[BUFFER_SIZE];
    
    // Inicializar todos los clientes a 0
    for (int i = 0; i < MAX_CLIENTS; i++) {
        client_sockets[i] = 0;
    }

    // Crear socket
    if ((server_fd = socket(AF_INET, SOCK_STREAM, 0)) == 0) {
        perror("socket falló");
        exit(EXIT_FAILURE);
    }

    // Configurar dirección del servidor
    address.sin_family = AF_INET;
    address.sin_addr.s_addr = INADDR_ANY;
    address.sin_port = htons(PORT);

    // Enlazar socket al puerto
    if (bind(server_fd, (struct sockaddr *)&address, sizeof(address)) < 0) {
        perror("bind falló");
        exit(EXIT_FAILURE);
    }

    // Escuchar conexiones entrantes
    if (listen(server_fd, 3) < 0) {
        perror("listen");
        exit(EXIT_FAILURE);
    }

    printf("Servidor escuchando en puerto %d...\n", PORT);

    while (1) {
        // Limpiar conjunto de descriptores
        FD_ZERO(&read_fds);
        FD_SET(server_fd, &read_fds);
        max_sd = server_fd;

        // Agregar sockets de clientes al conjunto
        for (int i = 0; i < MAX_CLIENTS; i++) {
            sd = client_sockets[i];
            if (sd > 0) FD_SET(sd, &read_fds);
            if (sd > max_sd) max_sd = sd;
        }

        // Esperar actividad
        activity = select(max_sd + 1, &read_fds, NULL, NULL, NULL);

        if ((activity < 0) && (errno != EINTR)) {
            printf("Error en select\n");
        }

        // Nueva conexión
        if (FD_ISSET(server_fd, &read_fds)) {
            socklen_t addrlen = sizeof(address);
            if ((new_socket = accept(server_fd, (struct sockaddr *)&address, &addrlen)) < 0) {
                perror("accept");
                exit(EXIT_FAILURE);
            }

            printf("Nueva conexión: socket fd %d, IP %s, puerto %d\n",
                   new_socket, inet_ntoa(address.sin_addr), ntohs(address.sin_port));

            // Agregar a la lista de clientes
            for (int i = 0; i < MAX_CLIENTS; i++) {
                if (client_sockets[i] == 0) {
                    client_sockets[i] = new_socket;
                    break;
                }
            }
        }

        // Revisar mensajes de clientes
        for (int i = 0; i < MAX_CLIENTS; i++) {
            sd = client_sockets[i];

            if (FD_ISSET(sd, &read_fds)) {
                if ((valread = read(sd, buffer, BUFFER_SIZE)) == 0) {
                    // Desconexión
                    getpeername(sd, (struct sockaddr*)&address, (socklen_t*)&addrlen);
                    printf("Cliente desconectado: IP %s, puerto %d\n",
                           inet_ntoa(address.sin_addr), ntohs(address.sin_port));
                    close(sd);
                    client_sockets[i] = 0;
                } else {
                    buffer[valread] = '\0';
                    printf("Mensaje recibido: %s", buffer);

                    // Reenviar mensaje a otros clientes
                    for (int j = 0; j < MAX_CLIENTS; j++) {
                        if (client_sockets[j] != 0 && client_sockets[j] != sd) {
                            send(client_sockets[j], buffer, strlen(buffer), 0);
                        }
                    }
                }
            }
        }
    }

    return 0;
}' > servidor_select.c

Compilo el código usando Bash C y verifico que el ejecutable esté disponible

In [ ]:
gcc servidor_select.c -o servidor_select | ls

Si abro la ventana terminal de esta forma se cerrará al finalizar la ejecución del servidor:

In [ ]:
gnome-terminal -- bash -c "./servidor_select; echo 'Presionar ENTER para cerrar...'; read"

## Servidor con select()

In [ ]:
echo '#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <arpa/inet.h>
#include <sys/socket.h>
#include <sys/select.h>

#define PORT 12345
#define BUFFER_SIZE 1024

int main() {
    int sock;
    struct sockaddr_in server_addr;
    char buffer[BUFFER_SIZE];
    fd_set read_fds;

    // Crear socket
    sock = socket(AF_INET, SOCK_STREAM, 0);
    if (sock < 0) {
        perror("socket");
        exit(EXIT_FAILURE);
    }

    // Configurar dirección del servidor
    server_addr.sin_family = AF_INET;
    server_addr.sin_port = htons(PORT);
    inet_pton(AF_INET, "127.0.0.1", &server_addr.sin_addr);

    // Conectarse al servidor
    if (connect(sock, (struct sockaddr *)&server_addr, sizeof(server_addr)) < 0) {
        perror("connect");
        exit(EXIT_FAILURE);
    }

    printf("Conectado al servidor. Puedes escribir mensajes:\n");

    while (1) {
        FD_ZERO(&read_fds);
        FD_SET(0, &read_fds);      // Entrada estándar
        FD_SET(sock, &read_fds);   // Socket

        if (select(sock + 1, &read_fds, NULL, NULL, NULL) < 0) {
            perror("select");
            exit(EXIT_FAILURE);
        }

        // Entrada del usuario
        if (FD_ISSET(0, &read_fds)) {
            fgets(buffer, BUFFER_SIZE, stdin);
            send(sock, buffer, strlen(buffer), 0);
        }

        // Mensaje del servidor
        if (FD_ISSET(sock, &read_fds)) {
            int valread = read(sock, buffer, BUFFER_SIZE - 1);
            if (valread <= 0) {
                printf("Desconectado del servidor.\n");
                break;
            }
            buffer[valread] = '\0';
            printf("Mensaje: %s", buffer);
        }
    }

    close(sock);
    return 0;
}' > cliente_select.c

Compilo el código usando Bash C y verifico que el ejecutable esté disponible:

In [ ]:
gcc cliente_select.c -o cliente_select | ls

Si abro la ventana terminal de esta forma se cerrará al finalizar la ejecución del servidor:

In [ ]:
gnome-terminal -- bash -c "./cliente_select; echo 'Presionar ENTER para cerrar...'; read"

### SERVIDOR PASO A PASO:
(1) Declaraciones y configuraciones iniciales

- #define PORT 12345
- #define MAX_CLIENTS 10
- #define BUFFER_SIZE 1024

- Puerto donde el servidor va a escuchar.
- Máximo de clientes simultáneos.
- Tamaño del buffer de mensajes.

int server_fd, new_socket, client_sockets[MAX_CLIENTS];
struct sockaddr_in address;
fd_set read_fds;

- server_fd: socket del servidor (para aceptar conexiones).
- new_socket: se usa cuando un cliente se conecta.
- client_sockets[]: array para guardar los sockets de los clientes.
- fd_set read_fds: conjunto de descriptores de archivo para pasar a select().

(2) Crear el socket

server_fd = socket(AF_INET, SOCK_STREAM, 0);

- Crea un socket TCP.

(3) Asignar dirección y puerto

address.sin_family = AF_INET;
address.sin_addr.s_addr = INADDR_ANY;
address.sin_port = htons(PORT);

- AF_INET: IPv4.
- INADDR_ANY: Acepta conexiones en cualquier interfaz de red.
- htons: convierte el número de puerto a formato de red.

(4) Enlazar y escuchar

bind(server_fd, (struct sockaddr *)&address, sizeof(address));
listen(server_fd, 3);

- Asocia el socket a una dirección IP y puerto.
- Empieza a escuchar conexiones entrantes.

(5) Bucle principal

while (1) {
    FD_ZERO(&read_fds); // limpiar conjunto
    FD_SET(server_fd, &read_fds); // añadir el socket servidor

- Limpiamos el conjunto de descriptores (FD_ZERO).
- Añadimos el descriptor del servidor (para detectar nuevas conexiones).

    for (int i = 0; i < MAX_CLIENTS; i++) {
        sd = client_sockets[i];
        if (sd > 0) FD_SET(sd, &read_fds);
    }

- Añadimos cada socket de cliente activo al conjunto read_fds.

    activity = select(max_sd + 1, &read_fds, NULL, NULL, NULL);

- select() espera hasta que alguno de los sockets esté listo para lectura.

(6) Si hay una nueva conexión

if (FD_ISSET(server_fd, &read_fds)) {
    new_socket = accept(...);
    client_sockets[i] = new_socket;
}

- Se acepta la nueva conexión.
- Se guarda el nuevo socket en el array client_sockets.

(7) Si un cliente envía un mensaje

if (FD_ISSET(sd, &read_fds)) {
    valread = read(sd, buffer, BUFFER_SIZE);
}

- Si read() devuelve 0, el cliente cerró la conexión.
- Si devuelve > 0, recibimos un mensaje y lo reenviamos a los demás clientes:

send(client_sockets[j], buffer, strlen(buffer), 0);

### CLIENTE PASO A PASO:
(1) Crear socket

sock = socket(AF_INET, SOCK_STREAM, 0);

- Creamos un socket TCP.

(2) Conectar al servidor

inet_pton(AF_INET, "127.0.0.1", &server_addr.sin_addr);
connect(sock, (struct sockaddr *)&server_addr, sizeof(server_addr));

- Convertimos la IP en texto a binario (inet_pton).
- Conectamos al socket del servidor.

(3) Bucle principal con select()

FD_SET(0, &read_fds);     // entrada estándar (teclado)
FD_SET(sock, &read_fds);  // socket de red
select(sock + 1, &read_fds, NULL, NULL, NULL);

- Monitorea si hay datos listos para leer desde:
- El teclado (stdin)
- El servidor (sock)

(4) Si el usuario escribe un mensaje

fgets(buffer, BUFFER_SIZE, stdin);
send(sock, buffer, strlen(buffer), 0);

- Leemos desde teclado y enviamos al servidor.

(5) Si el servidor envía un mensaje

valread = read(sock, buffer, BUFFER_SIZE - 1);

- Recibimos y mostramos el mensaje en pantalla.

### SERVIDOR
Notas importantes

- #include <sys/select.h> // select(), fd_set, FD_SET, etc.
- #include <sys/socket.h> // socket(), bind(), listen(), etc.
- #include <netinet/in.h> // struct sockaddr_in
- int server_fd = socket(AF_INET, SOCK_STREAM, 0);


FD_ZERO(&read_fds);              // Limpia el conjunto
FD_SET(server_fd, &read_fds);    // Agrega el socket del servidor
max_sd = server_fd;

- FD_ZERO(): limpia el conjunto read_fds.
- FD_SET(server_fd, &read_fds): agrega el descriptor del socket servidor.
- max_sd: guarda el descriptor más alto. Es requerido por select().

Agregar los sockets de clientes activos

for (int i = 0; i < MAX_CLIENTS; i++) {
    sd = client_sockets[i];
    if (sd > 0)
        FD_SET(sd, &read_fds);
    if (sd > max_sd)
        max_sd = sd;
}

- Recorre el array de sockets.
- Agrega los clientes activos a read_fds.
- Actualiza max_sd si el descriptor es mayor.

Llamar a select()

activity = select(max_sd + 1, &read_fds, NULL, NULL, NULL);

- Espera hasta que uno o más sockets estén listos para lectura.
- max_sd + 1: límite superior de descriptores a monitorear.
- Los otros parámetros son NULL: no monitorea escritura ni errores.

Nueva conexión entrante

if (FD_ISSET(server_fd, &read_fds)) {
    new_socket = accept(server_fd, ...);
}

- FD_ISSET() verifica si el servidor tiene una nueva conexión.
- accept() acepta la conexión y retorna un nuevo descriptor para el cliente.

Guardar el nuevo cliente

for (int i = 0; i < MAX_CLIENTS; i++) {
    if (client_sockets[i] == 0) {
        client_sockets[i] = new_socket;
        break;
    }
}

- Se guarda el socket del nuevo cliente en el array.

Verificar si un cliente envió un mensaje

if (FD_ISSET(sd, &read_fds)) {
    valread = read(sd, buffer, BUFFER_SIZE);
}

- FD_ISSET() verifica si el cliente ha enviado algo.
- read() lee los datos del socket.

Si el cliente cerró conexión

if (valread == 0) {
    close(sd);
    client_sockets[i] = 0;
}

- read() devuelve 0 si el cliente se desconectó.
- Se cierra el socket y se borra del array.

Reenviar mensaje a otros clientes

for (int j = 0; j < MAX_CLIENTS; j++) {
    if (client_sockets[j] != 0 && client_sockets[j] != sd)
        send(client_sockets[j], buffer, strlen(buffer), 0);
}

- Se envía el mensaje recibido a todos los clientes excepto al que lo envió.

## CLIENTE

Bucle de comunicación

while (1) {
    FD_ZERO(&read_fds);
    FD_SET(0, &read_fds);    // Entrada por teclado
    FD_SET(sock, &read_fds); // Socket con el servidor
    select(sock + 1, &read_fds, NULL, NULL, NULL);

- FD_SET(0): monitorea teclado (stdin) por si el usuario escribe.
- FD_SET(sock): monitorea el socket con el servidor.
- select() espera a que uno de ellos tenga datos para leer.

Leer del teclado y enviar al servidor

if (FD_ISSET(0, &read_fds)) {
    fgets(buffer, BUFFER_SIZE, stdin);
    send(sock, buffer, strlen(buffer), 0);
}

- Si el usuario escribió algo, lo enviamos al servidor con send().

Leer mensaje del servidor

if (FD_ISSET(sock, &read_fds)) {
    valread = read(sock, buffer, BUFFER_SIZE - 1);
    if (valread <= 0) break;
    buffer[valread] = '\0';
    printf("Mensaje: %s", buffer);
}

- Si el servidor envió algo, se muestra en pantalla.
- Si read() devuelve 0 o menor, el servidor cerró la conexión.

¿Por qué usamos FD_SET, FD_ZERO, FD_ISSET?

- FD_ZERO(&set): limpia el conjunto.
- FD_SET(fd, &set): agrega el descriptor fd al conjunto.
- FD_ISSET(fd, &set): verifica si fd tiene datos listos.

select() usa estos conjuntos para saber qué descriptor está listo para operar.

El uso de FD_SET() en el cliente es necesario para poder usar select() y detectar cuándo el usuario escribe algo (stdin) o cuándo llega un mensaje del servidor sin bloquearte en una sola operación.

En el cliente tenemos dos posibles fuentes de entrada:

- Teclado del usuario (stdin, que es descriptor 0)
- Socket de conexión al servidor (por ejemplo, descriptor 4)

Para saber cuál de esas fuentes tiene datos disponibles para leer, usamos select() y le pasamos un fd_set con esos dos descriptores:

FD_ZERO(&read_fds);         // Limpia el conjunto
FD_SET(0, &read_fds);       // Agrega teclado (stdin)
FD_SET(sock, &read_fds);    // Agrega socket con el servidor
select(sock + 1, &read_fds, NULL, NULL, NULL);

¿Y si no uso FD_SET(0, &read_fds)?

- El cliente no podría detectar si el usuario escribió algo.
- Solo sabría si el servidor envió un mensaje.
- Habría que usar fgets() o scanf() que bloquean el programa, y no se podría recibir mensajes mientras el usuario no escriba.

Ventajas de usar select() en el cliente con FD_SET()

- Permite que el cliente sea reactivo y no se bloquee.
- Permite recibir mensajes mientras el usuario escribe.
- Permite enviar mensajes apenas el usuario los escriba.
- Código más profesional.



## Cliente sin select() (versión simplificada y bloqueante)

In [ ]:
echo '#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <arpa/inet.h>

#define PORT 12345
#define BUFFER_SIZE 1024

int main() {
    int sock;
    struct sockaddr_in server_addr;
    char buffer[BUFFER_SIZE];

    // 1. Crear socket
    sock = socket(AF_INET, SOCK_STREAM, 0);
    if (sock == -1) {
        perror("Error al abrir el socket ...");
        exit(EXIT_FAILURE);
    }

    // 2. Configurar dirección del servidor
    server_addr.sin_family = AF_INET;
    server_addr.sin_port = htons(PORT);
    server_addr.sin_addr.s_addr = inet_addr("127.0.0.1");

    // 3. Conectar al servidor
    if (connect(sock, (struct sockaddr *)&server_addr, sizeof(server_addr)) < 0) {
        perror("Error al conectar");
        exit(EXIT_FAILURE);
    }

    printf("Conectado al servidor. Escribe mensajes y presiona Enter.\n");

    // 4. Bucle principal bloqueante
    while (1) {
        printf("> ");
        fgets(buffer, BUFFER_SIZE, stdin);  // BLOQUEA esperando entrada
        send(sock, buffer, strlen(buffer), 0);

        // Leer respuesta del servidor
        int valread = read(sock, buffer, BUFFER_SIZE - 1);
        if (valread <= 0) {
            printf("Conexión cerrada por el servidor.\n");
            break;
        }

        buffer[valread] = '\0';
        printf("Servidor: %s", buffer);
    }

    close(sock);
    return 0;
}' > cliente_bloqueante.c

Compilar y abrir termina según se explicó anteriormente

**Ejercicios a realizar durante la clase. Incluir los códigos dentro de este mismo Notebook**

B-1) Comparar cliente con select y bloqueante.   
B-2) Modificar el cliente bloqueante para que sea no bloqueante.   
B-3) Compara cliente con select y no bloqueante sin select.   
b-4) Conclusiones.   

## Consigna del TP 1

- Explicar qué es un socket y los diferentes tipos de sockets.
- Cuáles son las estructuras necesarias para operar con sockets en el modelo C-S y cómo se hace para ingresar los datos requeridos.
-- Explicar con un ejemplo.
- Explicar modo bloqueante y no bloqueante en sockets y cuáles son los “sockets calls” a los cuales se pueden aplicar estos modos.
-- Explicar las funciones necesarias.
- Describir el modelo C-S aplicado a un servidor con Concurrencia Real.
- Escribir un ejemplo en lenguaje C distinto del ya presentado.
- Cómo se cierra una conexión C-S ?. Métodos que eviten la pérdida de informacion.
- Probar el código del chat entre cliente y servidor. Cambiar tipo de socket y volver a probar.
- Presentar las respuestas en archivos Jupyter Notebook o PDF.

C-S = Cliente - Servidor  
- Bibliografía a consultar:  
-- Clases Teóricas.  
-- Brian "Beej Jorgensen" Hall. Beej's Guide to Network Programming: Using Internet Sockets. 2019.  
-- Douglas E. Comer, David L. Stevens. Internetworking With Tcp/Ip: Client-Server Programming and Applications. Prentice Hall. 2011.  